In [ ]:
import pandas as pd
import numpy as np
import scipy.stats
import math
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# tmp0_gr=tmp0.groupby(['msa','statefip'],as_index=False).agg({'id':sum})
# bmk0_gr=bmk0.groupby(['msa'],as_index=False).agg({'id':sum})
# msaid_state_lookup=tmp0_gr[['msa','statefip']].merge(bmk0_gr[['msa']],on='msa').groupby(['msa','statefip'],as_index=False).sum()
# msaid_state_lookup.to_excel('/Users/hannahkamen/Downloads/msaid_state_lookup.xlsx')

#####get MSA Identifier information
# msa_id=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/msa_identifier.dta')
# msa_vars=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/second_stage_dataset_cl.dta')
state_split=pd.read_excel('/Users/hannahkamen/Downloads/state_pop_educ_shares.xlsx')
sl=pd.read_excel('/Users/hannahkamen/Downloads/statelookup2.xlsx')

##### (2) get coordinate data

In [ ]:
###prepare dataset with centroid of every state

map_dta1=pd.read_csv('/Users/hannahkamen/Downloads/census_texas.csv')
map_dta2=pd.read_csv('/Users/hannahkamen/Downloads/census_ny.csv')
map_dta=map_dta1.append(map_dta2).reset_index()

###rename data
map_dta['centroid2']=map_dta['centroid1']+","+map_dta['centroid2']
map_dta['centroid1']=map_dta['FULL1_NAME']+","+map_dta['FULL2_NAME']
map_dta['FULL1_NAME']=map_dta['GEOID1']
map_dta['FULL2_NAME']=map_dta['GEOID2']

del map_dta['GEOID1']
del map_dta['GEOID2']


map_dta['GEOID1']=map_dta['level_1']
map_dta['GEOID2']=map_dta['Unnamed: 0']
map_dta=map_dta[['GEOID1','GEOID2','FULL1_NAME','FULL2_NAME','centroid1','centroid2']]
####get abbreviations

states=list(sl['abbrev'].unique())

state_coords=pd.DataFrame()
for s in states:
    tmp=map_dta[map_dta['FULL2_NAME'].str.contains(s)]
    tmp['state']=s
    state_coords=state_coords.append(tmp)
state_coords=state_coords.drop_duplicates(subset='state')
state_coords=state_coords[['state','FULL2_NAME','centroid2']]

In [ ]:
state_coords['id']= np.arange(len(state_coords))
state_coords['name']=state_coords['state']
state_coords['lat']=state_coords['centroid2'].apply(lambda x: x.split(',')[1].replace(")",'').strip())
state_coords['lon']=state_coords['centroid2'].apply(lambda x: x.split(',')[0].replace("(",'').replace("c",'').strip())
state_coords_lm=state_coords[['id','name','lat','lon']]
state_coords_lm.to_csv('/Users/hannahkamen/Downloads/flowmap_location_lookup.csv',index=False)


##### (2) map out migration flows

In [ ]:
#####import state pop shares by skilled and unskilled
state_educ=pd.read_excel('/Users/hannahkamen/Downloads/state_educ_shares.xlsx')
del state_educ['statefip']
state_educ=state_educ.merge(sl,on='state',how='inner')
state_educ=state_educ[['statefip','skl','unskl','state','abbrev','state_pop']]

In [ ]:
###these are the population shares changes from the first cycle of GAMS data inputs (after initial logit shock with temp)
###the shares caluclated here represent the population change from the previous iteration 
state_split=pd.read_excel('/Users/hannahkamen/Downloads/state_pop_educ_shares.xlsx')

msaid_state_lookup=pd.read_excel('/Users/hannahkamen/Downloads/msaid_state_lookup.xlsx')
sl=pd.read_excel('/Users/hannahkamen/Downloads/statelookup2.xlsx')
tmp0=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/projection_data_age2_01_wbmk_v2_0_ca_only.dta')
tmp0=tmp0.merge(sl,on='statefip')


In [ ]:
# ####store in csv

master_map_unskl2=tmp0[tmp0['coll']==0]
master_map_skl2=tmp0[tmp0['coll'] ==1]

del tmp0

master_map_unskl2.to_csv('/Users/hannahkamen/Downloads/map_data_unskl2_ca_only.csv')
del master_map_unskl2
master_map_skl2.to_csv('/Users/hannahkamen/Downloads/map_data_skl2_ca_only.csv')
del master_map_skl2

In [ ]:
master_map_unskl2=pd.read_csv('/Users/hannahkamen/Downloads/map_data_unskl2_ca_only.csv')
master_map_skl2=pd.read_csv('/Users/hannahkamen/Downloads/map_data_skl2_ca_only.csv')


#### Unskilled Map

In [ ]:
###### prep data for map

#get sum of shares in all other states for people who chose states 1-44
#get ids that live in state s currentlly
unskl=pd.DataFrame()
for s in master_map_unskl2['state'].unique():
    living_now=[]
    tmp1=master_map_unskl2[(master_map_unskl2['state']==s) & (master_map_unskl2['chosen']==1)]
    ####list of ids currently living in state s
    living_now.append(list(tmp1['id'].unique())[0])
    ###limit dataframe to the ids of people living in state s
    tmp2=master_map_unskl2[master_map_unskl2['id'].isin(living_now)]
    ####now groupby shares across all states
    tmp3=tmp2.groupby(['state'],as_index=False).agg({'share0':sum,'shareb':sum})
    ###tag origin state
    tmp3['living_flag']=s
    unskl=unskl.append(tmp3)
    
######merge with state populaiton
unskl=unskl.merge(state_split,left_on='living_flag',right_on='state',how='inner')
unskl=unskl.rename(columns={'state_x':'moving_to'})

######get share difference between current  iteration and benchmark
unskl['share_diff']=unskl['share0']-unskl['shareb']

In [ ]:
len(master_map_unskl2[master_map_unskl2['chosen']==1]['state'].unique())

In [ ]:
###### prep data for map

#get sum of shares in all other states for people who chose states 1-44
#get ids that live in state s currentlly
skl=pd.DataFrame()
for s in master_map_skl2['state'].unique():
    living_now=[]
    tmp1=master_map_skl2[(master_map_skl2['state']==s) & (master_map_skl2['chosen']==1)]
    ####list of ids currently living in state s
    living_now.append(list(tmp1['id'].unique())[0])
    ###limit dataframe to the ids of people living in state s
    tmp2=master_map_skl2[master_map_skl2['id'].isin(living_now)]
    ####now groupby shares across all states
    tmp3=tmp2.groupby(['state'],as_index=False).agg({'share0':sum,'shareb':sum})
    ###tag origin state
    tmp3['living_flag']=s
    skl=skl.append(tmp3)
    
######merge with state populaiton
skl=skl.merge(state_split,left_on='living_flag',right_on='state',how='inner')
skl=skl.rename(columns={'state_x':'moving_to'})

######get share difference between current  iteration and benchmark
skl['share_diff']=skl['share0']-skl['shareb']

In [ ]:
####read in pct changes by state and education
le0=pd.read_csv('/Users/hannahkamen/Downloads/le0_shock0_v2_test2_ncollege_only.csv')

le0=le0[['r','sk','skill_shr']]
le0=le0.drop_duplicates(subset=['r','sk'])
le0=le0.rename(columns={'sk':'educ_id','skill_shr':'final_change','r':'abbrev'})
le0=le0.merge(sl,on='abbrev')
le0_skl=le0[le0['educ_id']=='skl']
le0_unskl=le0[le0['educ_id']=='unskl']

In [ ]:
#####prepare pop map data for skilled entrance
all_skl_entrance=pd.DataFrame()

for s in le0_skl[le0_skl['final_change']>0]['state'].unique():
    ###for each entrance state, calculate shares by origin state that make up total change in the positive share diff
    master_map_skl_lm=skl[((skl['moving_to']==s) & (skl['share_diff']>0))]
    #####get totals 
    master_map_skl_lm_totalinflows=master_map_skl_lm.groupby('moving_to').agg({'share_diff':sum})
    ###rename and merge
    master_map_skl_lm_totalinflows=master_map_skl_lm_totalinflows.rename(columns={'share_diff':'share_diff_tot'})
    master_map_skl_lm=master_map_skl_lm.merge(master_map_skl_lm_totalinflows,on='moving_to')
    
    master_map_skl_lm['inflow_share']= master_map_skl_lm['share_diff']/master_map_skl_lm['share_diff_tot']
    
    master_map_skl_lm=master_map_skl_lm[['moving_to','living_flag','inflow_share','share_diff_tot']]
    all_skl_entrance=all_skl_entrance.append(master_map_skl_lm)
    
all_skl_entrance=all_skl_entrance.merge(state_educ,left_on='moving_to',right_on='state').merge(le0_skl,on='state')
all_skl_entrance['raw_pop_change_moving_to']=all_skl_entrance['final_change']*all_skl_entrance['state_pop']*all_skl_entrance['skl']*all_skl_entrance['inflow_share']

##merge in pop info

all_skl_entrance=all_skl_entrance.rename(columns={'abbrev_x':'dest'})
all_skl_entrance=all_skl_entrance.merge(sl[['state','abbrev']],left_on='living_flag',right_on='state')
all_skl_entrance=all_skl_entrance.rename(columns={'abbrev':'origin'})

all_skl_entrance=all_skl_entrance.merge(state_coords_lm,left_on='dest',right_on='name')
del all_skl_entrance['dest']
all_skl_entrance['dest']=all_skl_entrance['id']
del all_skl_entrance['id']
all_skl_entrance=all_skl_entrance.merge(state_coords_lm,left_on='origin',right_on='name')
del all_skl_entrance['origin']


# all_skl_entrance=all_skl_entrance.merge(state_educ,left_on='moving_to',right_on='state').merge(total_skilled_pop_changes,left_on='moving_to',right_on='statefip')
all_skl_entrance['origin']=all_skl_entrance['id']
all_skl_entrance['count']=all_skl_entrance['raw_pop_change_moving_to']

all_skl_entrance_lm=all_skl_entrance[['origin','dest','count']]
all_skl_entrance_lm['count']=all_skl_entrance_lm['count']/14412444.2
all_skl_entrance_lm.to_csv('/Users/hannahkamen/Downloads/skilled_entrance_ca_only.csv',index=False)



In [ ]:
#####prepare pop map data for skilled entrance
all_unskl_entrance=pd.DataFrame()

for s in le0_unskl[le0_unskl['final_change']>0]['state'].unique():
    ###for each entrance state, calculate shares by origin state that make up total change in the positive share diff
    master_map_unskl_lm=unskl[((unskl['moving_to']==s) & (unskl['share_diff']>0))]
    #####get totals 
    master_map_unskl_lm_totalinflows=master_map_unskl_lm.groupby('moving_to').agg({'share_diff':sum})
    ###rename and merge
    master_map_unskl_lm_totalinflows=master_map_unskl_lm_totalinflows.rename(columns={'share_diff':'share_diff_tot'})
    master_map_unskl_lm=master_map_unskl_lm.merge(master_map_unskl_lm_totalinflows,on='moving_to')
    
    master_map_unskl_lm['inflow_share']= master_map_unskl_lm['share_diff']/master_map_unskl_lm['share_diff_tot']
    
    master_map_unskl_lm=master_map_unskl_lm[['moving_to','living_flag','inflow_share','share_diff_tot']]
    all_unskl_entrance=all_unskl_entrance.append(master_map_unskl_lm)
    
all_unskl_entrance=all_unskl_entrance.merge(state_educ,left_on='moving_to',right_on='state').merge(le0_unskl,on='state')
all_unskl_entrance['raw_pop_change_moving_to']=all_unskl_entrance['final_change']*all_unskl_entrance['state_pop']*all_unskl_entrance['unskl']*all_unskl_entrance['inflow_share']

##merge in pop info

all_unskl_entrance=all_unskl_entrance.rename(columns={'abbrev_x':'dest'})
all_unskl_entrance=all_unskl_entrance.merge(sl[['state','abbrev']],left_on='living_flag',right_on='state')
all_unskl_entrance=all_unskl_entrance.rename(columns={'abbrev':'origin'})

all_unskl_entrance=all_unskl_entrance.merge(state_coords_lm,left_on='dest',right_on='name')
del all_unskl_entrance['dest']
all_unskl_entrance['dest']=all_unskl_entrance['id']
del all_unskl_entrance['id']
all_unskl_entrance=all_unskl_entrance.merge(state_coords_lm,left_on='origin',right_on='name')
del all_unskl_entrance['origin']


# all_unskl_entrance=all_unskl_entrance.merge(state_educ,left_on='moving_to',right_on='state').merge(total_skilled_pop_changes,left_on='moving_to',right_on='statefip')
all_unskl_entrance['origin']=all_unskl_entrance['id']
all_unskl_entrance['count']=all_unskl_entrance['raw_pop_change_moving_to']

all_unskl_entrance_lm=all_unskl_entrance[['origin','dest','count']]

all_unskl_entrance_lm['count']=all_unskl_entrance_lm['count']/14412444.2

all_unskl_entrance_lm.to_csv('/Users/hannahkamen/Downloads/unskilled_entrance_ca_only.csv',index=False)

